# Cloropleth map

## Objetivo

* Crear mapa de cloropletas

## Importar librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importar geopandas, shapely.geometry y plotly

In [2]:
from geopy.geocoders import Nominatim
# geopy-python: librería para geocoding
# geocoders: lo mismo que geopy
import geopandas as gpd
from shapely.geometry import Point
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Usando dataset ../../Data/02ParaLimpiar/02desastres_paralimpiar.csv

## Cargar datos

In [3]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', encoding='utf-8', delimiter=';', engine='python')

## Verificar carga de dataset

##  Resumen básico shape, info, head

In [4]:
df.shape
# Resultado: 16636 filas y 20 columnas)

(16636, 26)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dis No             16636 non-null  object 
 1   Year               16636 non-null  int64  
 2   Seq                16636 non-null  int64  
 3   Disaster Subgroup  16636 non-null  object 
 4   Disaster Type      16636 non-null  object 
 5   Disaster Subtype   13313 non-null  object 
 6   Country            16636 non-null  object 
 7   ISO                16636 non-null  object 
 8   Region             16636 non-null  object 
 9   Continent          16636 non-null  object 
 10  Location           14825 non-null  object 
 11  Origin             4085 non-null   object 
 12  Associated Dis     3593 non-null   object 
 13  Dis Mag Value      5064 non-null   float64
 14  Dis Mag Scale      15416 non-null  object 
 15  Latitude           2775 non-null   object 
 16  Longitude          277

In [6]:
df.head(1)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN


In [7]:
df['Latitude'][:40]

0        NaN
1        NaN
2         14
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9      32.04
10     33.05
11    1.51 N
12       NaN
13       NaN
14       NaN
15      38.5
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21     44.05
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       9.8
29        36
30       NaN
31       NaN
32       NaN
33       NaN
34       NaN
35       NaN
36       NaN
37    -3.924
38       NaN
39       NaN
Name: Latitude, dtype: object

In [8]:
df['Longitude'][:40]

0         NaN
1         NaN
2         -91
3         NaN
4         NaN
5         NaN
6         NaN
7         NaN
8         NaN
9       76.16
10       71.4
11    78.46 W
12        NaN
13        NaN
14        NaN
15       69.9
16        NaN
17        NaN
18        NaN
19        NaN
20        NaN
21       6.14
22        NaN
23        NaN
24        NaN
25        NaN
26        NaN
27        NaN
28        -84
29          4
30        NaN
31        NaN
32        NaN
33        NaN
34        NaN
35        NaN
36        NaN
37     101.82
38        NaN
39        NaN
Name: Longitude, dtype: object

# Filtros para Location y geolocalización

In [9]:
# Categorías geográficas
geographical_data = df[['Location', 'ISO', 'Country', 'Region', 'Continent']]

# Categorías de magnitud
magnitude_data = df[['Dis Mag Scale', 'Dis Mag Value']]

# Categorías de geolocalización
geolocation_data = df[['Latitude', 'Longitude']]

# Categorías de fecha de Earthquake
datetime_data = df[['Start Day', 'Start Month', 'Start Year', 'End Day', 'End Month', 'End Year']]

# Limpiar coordenadas

## Función para limpiar coordenadas

In [10]:
# # Convertir 'Latitude' y 'Longitude' a string
# df['Latitude'] = df['Latitude'].astype(str)
# df['Longitude'] = df['Longitude'].astype(str)

# # # Aplicar valores negativos donde Latitude y Longitude contengan S o W
# # df.loc[df['Latitude'].str.contains('S'), 'Latitude'] *= -1
# # df.loc[df['Longitude'].str.contains('W'), 'Longitude'] *= -1

# # Limpiar las coordenadas latitude y longitude
# # df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# # df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)


# # Eliminar puntos finales
# df['Latitude'] = df['Latitude'].str.rstrip('.')
# df['Longitude'] = df['Longitude'].str.rstrip('.')

# # Cambiar strings vacíos a NaN
# df['Latitude'] = df['Latitude'].replace('', np.nan)
# df['Longitude'] = df['Longitude'].replace('', np.nan)

# # Convertir coordenadas válidas a float
# df['Latitude'] = df['Latitude'].astype(float)
# df['Longitude'] = df['Longitude'].astype(float)

# # Identificar valores anómalos a partir de los grados
# anomalous_lat = df['Latitude'].abs() > 90
# anomalous_lon = df['Longitude'].abs() > 180

# # Configurar valores anómalos a NaN
# df.loc[anomalous_lat, 'Latitude'] = np.nan
# df.loc[anomalous_lon, 'Longitude'] = np.nan

## Lirería re

\d

Para los patrones de Unicode (str):
Coincide con cualquier dígito decimal de Unicode (es decir, cualquier carácter de la categoría de caracteres de Unicode [Nd]). Esto incluye a [0-9], y también muchos otros caracteres de dígitos. Si se usa el indicador ASCII, sólo coincide con [0-9].

re.sub(pattern, repl, string, count=0, flags=0)¶
Retorna la cadena obtenida reemplazando las ocurrencias no superpuestas del pattern («patrón») en la string («cadena») por el reemplazo de repl. Si el patrón no se encuentra, se retorna string sin cambios. repl puede ser una cadena o una función; si es una cadena, cualquier barra inversa escapada en ella es procesada. Es decir, \n se convierte en un carácter de una sola línea nueva, \r se convierte en un retorno de carro, y así sucesivamente. Los escapes desconocidos de las letras ASCII se reservan para un uso futuro y se tratan como errores. Otros escapes desconocidos como \& no se utilizan. Las referencias inversas, como \6, se reemplazan por la subcadena que corresponde al grupo 6 del patrón. Por ejemplo:

>>>
re.sub(r'def\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*\(\s*\):',
       r'static PyObject*\npy_\1(void)\n{',
       'def myfunc():')
'static PyObject*\npy_myfunc(void)\n{'
Si repl es una función, se llama para cada ocurrencia no superpuesta de pattern. La función toma un solo argumento objeto match, y retorna la cadena de sustitución. Por ejemplo:

>>>
def dashrepl(matchobj):
    if matchobj.group(0) == '-': return ' '
    else: return '-'
re.sub('-{1,2}', dashrepl, 'pro----gram-files')
'pro--gram files'
re.sub(r'\sAND\s', ' & ', 'Baked Beans And Spam', flags=re.IGNORECASE)
'Baked Beans & Spam'
El patrón puede ser una cadena o un objeto patrón.

El argumento opcional count («recuento») es el número máximo de ocurrencias de patrones a ser reemplazados; count debe ser un número entero no negativo. Si se omite o es cero, todas las ocurrencias serán reemplazadas. Las coincidencias vacías del patrón se reemplazan sólo cuando no están adyacentes a una coincidencia vacía anterior, así que sub('x*', '-', 'abxd') retorna '-a-b--d-'.

En los argumentos repl de tipo cadena, además de los escapes de caracteres y las referencias inversas descritas anteriormente, \g<name> usará la subcadena coincidente con el grupo llamado name, como se define en la sintaxis (?P<name>...). \g<number> utiliza el número de grupo correspondiente; \g<2> es por lo tanto equivalente a \2, pero no es ambiguo en un reemplazo como sucede con \g<2>0. \20 se interpretaría como una referencia al grupo 20, no como una referencia al grupo 2 seguido del carácter literal '0'. La referencia inversa \g<0> sustituye en toda la subcadena coincidente con la RE.

Distinto en la versión 3.1: Se añadió el argumento de los indicadores opcionales.

Distinto en la versión 3.5: Los grupos no coincidentes son reemplazados por una cadena vacía.

Distinto en la versión 3.6: Los escapes desconocidos en el pattern que consisten en '\' y una letra ASCII ahora son errores.

Distinto en la versión 3.7: Los escapes desconocidos en repl que consisten en '\' y una letra ASCII ahora son errores.

Distinto en la versión 3.7: Las coincidencias vacías para el patrón se reemplazan cuando están adyacentes a una coincidencia anterior no vacía.

Obsoleto desde la versión 3.11: Group id containing anything except ASCII digits. Group name containing characters outside the ASCII range (b'\x00'-b'\x7f') in bytes replacement strings.

re.subn(pattern, repl, string, count=0, flags=0)
Realiza la misma operación que sub(), pero retorna una tupla (new_string, number_of_subs_made).

Distinto en la versión 3.1: Se añadió el argumento de los indicadores opcionales.

Distinto en la versión 3.5: Los grupos no coincidentes son reemplazados por una cadena vacía.

re.escape(pattern)
Caracteres de escape especiales en pattern (» patrón»). Esto es útil si quieres hacer coincidir una cadena literal arbitraria que puede tener metacaracteres de expresión regular en ella. Por ejemplo:

>>>
print(re.escape('https://www.python.org'))
https://www\.python\.org

legal_chars = string.ascii_lowercase + string.digits + "!#$%&'*+-.^_`|~:"
print('[%s]+' % re.escape(legal_chars))
[abcdefghijklmnopqrstuvwxyz0123456789!\#\$%\&'\*\+\-\.\^_`\|\~:]+

operators = ['+', '-', '*', '/', '**']
print('|'.join(map(re.escape, sorted(operators, reverse=True))))
/|\-|\+|\*\*|\*
Esta función no debe usarse para la cadena de reemplazo en sub() y subn(), sólo deben escaparse las barras inversas. Por ejemplo:

>>>
digits_re = r'\d+'
sample = '/usr/sbin/sendmail - 0 errors, 12 warnings'
print(re.sub(digits_re, digits_re.replace('\\', r'\\'), sample))
/usr/sbin/sendmail - \d+ errors, \d+ warnings
Distinto en la versión 3.3: El carácter de '_' ya no se escapa.

Distinto en la versión 3.7: Sólo se escapan los caracteres que pueden tener un significado especial en una expresión regular. Como resultado, '!', '"', '%', "'", ',', '/', ':', ';', '<', '=', '>', '@' y "`" ya no se escapan.

In [11]:
# Convertir 'Latitude' y 'Longitude' a string
df['Latitude'] = df['Latitude'].astype(str)
df['Longitude'] = df['Longitude'].astype(str)
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Aplicar valores negativos donde Latitude y Longitude contengan S o W
# df.loc[df['Latitude'].str.contains('S'), 'Latitude'] *= -1
# df.loc[df['Longitude'].str.contains('W'), 'Longitude'] *= -1
# print(df['Latitude'][:40])
# print(df['Longitude'][:40])

# Limpiar las coordenadas latitude y longitude
# df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.--]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.--]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Eliminar puntos finales
df['Latitude'] = df['Latitude'].str.rstrip('.')
df['Longitude'] = df['Longitude'].str.rstrip('.')
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Cambiar strings vacíos a NaN
df['Latitude'] = df['Latitude'].replace('', np.nan)
df['Longitude'] = df['Longitude'].replace('', np.nan)
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Convertir coordenadas válidas a float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Identificar valores anómalos a partir de los grados
anomalous_lat = df['Latitude'].abs() > 90
anomalous_lon = df['Longitude'].abs() > 180
print(df['Latitude'][:40])
print(df['Longitude'][:40])

# Configurar valores anómalos a NaN
df.loc[anomalous_lat, 'Latitude'] = np.nan
df.loc[anomalous_lon, 'Longitude'] = np.nan
print(df['Latitude'][:40])
print(df['Longitude'][:40])

0        nan
1        nan
2         14
3        nan
4        nan
5        nan
6        nan
7        nan
8        nan
9      32.04
10     33.05
11    1.51 N
12       nan
13       nan
14       nan
15      38.5
16       nan
17       nan
18       nan
19       nan
20       nan
21     44.05
22       nan
23       nan
24       nan
25       nan
26       nan
27       nan
28       9.8
29        36
30       nan
31       nan
32       nan
33       nan
34       nan
35       nan
36       nan
37    -3.924
38       nan
39       nan
Name: Latitude, dtype: object
0         nan
1         nan
2         -91
3         nan
4         nan
5         nan
6         nan
7         nan
8         nan
9       76.16
10       71.4
11    78.46 W
12        nan
13        nan
14        nan
15       69.9
16        nan
17        nan
18        nan
19        nan
20        nan
21       6.14
22        nan
23        nan
24        nan
25        nan
26        nan
27        nan
28        -84
29          4
30        nan
31        nan
32

NameError: name 're' is not defined

La función re.sub() se usa para eliminar caracteres que no son dígitos de las coordenadas.
'[^\d.-]' es un patrón de expresión regular que coincide con cualquier carácter que no sea un dígito, un punto (punto decimal) o un guión (signo negativo).
x.split('.', 1)[0] divide la cadena en la primera aparición del punto decimal y conserva solo la parte anterior. Esto garantiza que solo se conserve la parte válida del valor de la coordenada.
La función str.rstrip('.') se utiliza para eliminar los puntos finales de las coordenadas.

### Explicación del fragmento de código para identificar valores anómalos de latitud y longitud en el DataFrame.

#### anomalous_lat = df['Latitud'].abs() > 90:

* df['Latitude'].abs() devuelve los valores absolutos de la columna 'Latitude'.
* > 90 comprueba si los valores de latitud absoluta son superiores a 90.
Esta expresión crea una serie booleana (anomalous_lat) donde True indica que la latitud es anómala (fuera del rango válido de -90 a 90 grados).

#### anomalous_lon = df['Longitud'].abs() > 180:

df['Longitud'].abs() devuelve los valores absolutos de la columna 'Longitud'.
> 180 comprueba si los valores de longitud absoluta son superiores a 180.
Esta expresión crea una serie booleana (anomalous_lon) donde True indica que la longitud es anómala (fuera del rango válido de -180 a 180 grados).
Las series booleanas anómalas_lat y anómalas_lon resultantes tendrán valores verdaderos para las filas donde los valores de latitud o longitud están fuera del rango válido, y falsos para las filas con coordenadas válidas.

Se pueden usar estas series booleanas para filtrar o manejar las filas con coordenadas anómalas en su marco de datos. Por ejemplo, se pueden eliminar las filas con coordenadas anómalas utilizando el siguiente código:

df_cleaned_coords = df[~(anomalous_lat | anomalous_lon)]

El operador ~ niega la serie booleana, por lo que ~(anomalous_lat | anomalou_lon) selecciona filas donde ni la latitud ni la longitud son anómalas. El DataFrame resultante, df_cleaned_coords, contendrá solo las filas con coordenadas válidas.

## Tests de verificación de limpieza de coordenadas

In [ ]:
print(df['Latitude'])
print(df['Longitude'])

In [ ]:
print(df.head())

In [ ]:
print(df.dtypes)

In [ ]:
print(df['Latitude'].unique())
print(df['Longitude'].unique())

In [ ]:
# Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
print(df.loc[68, 'Latitude'])
print(df.loc[68, 'Longitude'])

### Valores faltantes

In [ ]:
df[['Longitude', 'Latitude']].isnull().sum()

# Visualizaciones para comprobar coordenadas

## Boxplot para identificar outliers de Latitude y Longitude

<img src="../../Documentacion/CoordenadasLatitudesLongitudes.webp">

In [ ]:
sns.boxplot(df['Longitude'])

La mayor concentración de desastres naturales a partir de Longitude se ubica entre los 10º y 100º, afectando a países de África y Eurasia.
Los outliers para Longitude se concentran entre los -125º y los -180º, es decir, Pacífico Occidental.

In [ ]:
sns.boxplot(df['Latitude'])

# Coropleth map

### Crear un GeoDataFrame a partir del DataFrame

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))

### Crear un geojson

In [ ]:
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

### Guardar geojson

In [ ]:
gdf.to_file('coordenadas.geojson', driver='GeoJSON')

### Cargar el geojson

In [ ]:
gdf = gpd.read_file('coordenadas.geojson')

### Mapa de coropletas para Magnitud

In [ ]:
# Combinar 'Disaster Mag Scale' and 'Disaster Mag Value' en una columna compuesta
df['Magnitude'] = df['Dis Mag Scale'] + ' ' + df['Dis Mag Value'].astype(str)

# Crear GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Crear el choropleth usando la columna compuesta 'Magnitude'
fig = make_subplots(specs=[[{"type": "choropleth"}]])
fig.add_trace(
    go.Choroplethmapbox(
        geojson=gdf.geometry.__geo_interface__,
        locations=gdf.index,
        z=gdf['Magnitude'],
        colorscale='Viridis',
        marker_opacity=0.7,
        marker_line_width=0,
        featureidkey='properties.index',  # Especificar la característica id key
        colorbar=dict(title='Magnitude')  # Especificar el título de colorbar
    )
)
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=2,
    mapbox_center={"lat": 0, "lon": 0}
)
fig.show()

Da error de traceback call y tarda demasiado.

### Mapa de coropletas para Disaster Subtype

In [ ]:
# Crear GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Crear el choropleth usando la columna compuesta 'Magnitude'
fig = make_subplots(specs=[[{"type": "choropleth"}]])
fig.add_trace(
    go.Choroplethmapbox(
        geojson=gdf.geometry.__geo_interface__,
        locations=gdf.index,
        z=gdf['Disaster Subtype'],
        colorscale='Viridis',
        # marker_opacity=0.7,
        # marker_line_width=0,
        featureidkey='properties.index',  # Especificar la característica id key
        colorbar=dict(title='Magnitude')  # Especificar el título de colorbar
    )
)
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=2,
    mapbox_center={"lat": 0, "lon": 0}
)
fig.show()

In [ ]:
px.choropleth(df, locations='iso', color='Magnitude', hover_name='Country')